In [1]:
# 导入库
from transformers import AutoTokenizer,AutoModelForCausalLM,TrainingArguments,Trainer
from datasets import load_dataset
import pprint
import torch
print(torch.__version__)

d:\ProgramData\miniconda3\envs\test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.2.0+cpu


In [2]:
# 配置模型、数据
model_name="EleutherAI/pythia-70m" #hugging face模型
dataset_path="lamini/lamini_docs" #hugging face数据

In [3]:
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token # 以0填充

#加载模型
base_model=AutoModelForCausalLM.from_pretrained(model_name)

# 输出模型使用的device
device_count=torch.cuda.device_count()

if device_count>0:
    device=torch.device("cuda")
else:
    device = torch.device('cpu')
print("设备类型：",device)
print("模型结构\n",base_model)


d:\ProgramData\miniconda3\envs\test\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


设备类型： cpu
模型结构
 GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bia

In [4]:
# 加载训练集测试集数据
train_dataset=load_dataset(path=dataset_path,split="train")
test_dataset=load_dataset(path=dataset_path,split="test")

In [5]:
# 定义推理函数,根据问题回答的函数,相当于机器学习中的预测
## 训练之前先预测看看效果
def inference(text,model,tokenizer,max_input_tokens=1000,max_output_tokens=100):
    # Tokenize 文本分词，生成分词编码，类型为pytorch的张量tensor,tokens
    input_ids = tokenizer.encode(text,return_tensors='pt',truncation=True,max_length=max_input_tokens)

    # Generate 模型根据这些编码tensor的tokens，生成内容的tokens，模型生成的最大的词元数量为100
    device = model.device
    generated_tokens_with_prompt=model.generate(
        input_ids=input_ids.to(device),
        max_length=max_output_tokens # 生成更过内容时，需要更多时间
    )

    # Decode 对模型生成的tokens用分词器解码
    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt)
    
    # Strip the prompt # 删除初始的提示词，让它只返回结果
    generated_text_answer=generated_text_with_prompt[0][len(text):]
    
    return generated_text_answer


In [6]:
# 测试集测试
test_text=test_dataset[0]["question"] # 取第0行的question列
print("问题为：")
print(test_text)
print("默认回答为：")
print(test_dataset[0]["answer"])

print("模型的推理结果为：")
pprint.pprint(inference(test_text,base_model,tokenizer=tokenizer))


问题为：
Can Lamini generate technical documentation or user manuals for software projects?
默认回答为：
Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.
模型的推理结果为：


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


('\n'
 '\n'
 'I have a question about the following:\n'
 '\n'
 'How do I get the correct documentation to work?\n'
 '\n'
 'A:\n'
 '\n'
 'I think you need to use the following code:\n'
 '\n'
 'A:\n'
 '\n'
 'You can use the following code to get the correct documentation.\n'
 '\n'
 'A:\n'
 '\n'
 'You can use the following code to get the correct documentation.\n'
 '\n'
 'A:\n'
 '\n'
 'You can use the following')


In [7]:
# 定义微调训练的参数
## 定义输出模型样式
max_steps=3 # 定义最大步数
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir=trained_model_name

## 定义训练参数
training_args=TrainingArguments(

    # 学习率
    learning_rate=1.0e-5, #解释: 学习率是优化器调整模型权重时的步长大小。一个较小的学习率意味着每次更新的步伐较小，从而使训练过程更加稳定，但也可能导致训练速度变慢。
                          #设置: 1.0e-5，这是一个非常小的学习率，通常用于微调模型时，以确保模型在小范围内更新，避免对预训练模型的权重造成较大修改。

    # epochs数，对整个数据集遍历的次数
    num_train_epochs=1,   #解释: 最大训练轮次，即整个训练集将被迭代的次数。一个 epoch 代表模型经过一次完整的训练数据集。
                          #设置: 1，意味着模型将整个训练数据集遍历一次。这通常用于实验或在数据量较小的情况下。

    # 对模型的更新次数
    max_steps=max_steps,  #解释: 最大训练步数，即模型在训练期间所处理的批次数。这个参数与 max_train_epochs 互斥使用，通常用来设置训练的步数上限。
                          #设置: 这个参数的具体值没有在这里列出，但它需要与 max_train_epochs 一起考虑，以避免重复计算。

    # 每个设备上训练时的批量大小。批量大小决定了每次梯度更新时使用多少样本。
    per_device_train_batch_size=1,  #解释: 每个设备上训练时的批量大小。批量大小决定了每次梯度更新时使用多少样本。
                                    #设置: 1，这表示每个设备每次训练只处理一个样本，这可能会导致训练速度较慢，但可以在显存较小的情况下进行训练。

    # 模型快照的保存路径
    output_dir=output_dir,

    # 其他参数
    overwrite_output_dir=False, #覆写输出目录的内容
    disable_tqdm=False, #关闭过程进度条
    eval_steps=120, #每隔120步进行一次评估
    save_steps=120, #每隔120步进行一次保存
    warmup_steps=1, #预热步骤数。在训练初期，学习率会逐渐增加到设定的学习率，然后再进行正常训练。设置: 1，表示训练的前 1 步进行学习率预热。
    per_device_eval_batch_size=1,  # 每个设备上评估时的批量大小。
    eval_strategy="steps", # 评估策略，按照步数进行评估
    logging_strategy="steps", #按照步数，记录日志
    logging_steps=1, # 每一步记录一次
    optim="adafactor",#使用 Adafactor 优化器，这是一种用于大规模模型的优化算法。
    gradient_accumulation_steps=4, #解释: 梯度累积步骤数。在这个步骤数内累积梯度，然后进行一次参数更新。这有助于在显存较小的情况下使用较大的有效批量大小。设置: 4，表示每积累 4 个步骤的梯度后再进行一次更新
    gradient_checkpointing=False,#解释: 是否启用梯度检查点技术。这种技术可以节省显存，但会增加计算时间。设置: False，表示不启用梯度检查点。

    # 设置提前停止的参数
    load_best_model_at_end=True, #解释: 是否在训练结束时加载最佳模型。通常与提前停止机制一起使用。设置: True，表示在训练结束时自动加载在评估指标上表现最好的模型。
    save_total_limit=1, #解释: 最大保存检查点数量。如果设置了这个限制，系统将保留最近的检查点，而删除较早的检查点。设置: 1，表示只保留一个最新的检查点，删除旧的检查点以节省存储空间。
    metric_for_best_model="eval_loss", # 解释: 用于确定最佳模型的评估指标。设置: "eval_loss"，表示以评估损失作为选择最佳模型的依据。
    greater_is_better=False #解释: 指定是否指标越大越好。如果设为 True，则较高的指标值表示模型性能更好；如果设为 False，则较低的指标值表示模型性能更好。设置: False，表示评估损失越小越好。
)

In [8]:
# 训练模型的配置
training_config={
    "model":{
        "pretrained_name":model_name,
        "max_length":2048
    },
    "dataset":{
        "path":dataset_path
    },
    "verbose":True
}

# 展示模型占用内存空间
model_flops=(
    base_model.floating_point_ops(
        {
            "input_ids":torch.zeros(
                (1,training_config["model"]["max_length"])
            )
        }
    )
    *training_args.gradient_accumulation_steps
)

# 模型内存占用
print("Memory footprint",base_model.get_memory_footprint()/1e9,"GB")
print("Flops",model_flops/1e9,"GFLOPs")


Memory footprint 0.3084454 GB
Flops 2195.667812352 GFLOPs


In [9]:
# 模型训练
trainer=Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs


In [10]:
# 保存模型
save_dir=f'{output_dir}/final'

trainer.save_model(save_dir)
print("模型保存至：",save_dir)

 33%|███▎      | 1/3 [00:02<00:04,  2.30s/it]

{'loss': 4.1562, 'grad_norm': 76.75840759277344, 'learning_rate': 1e-05, 'epoch': 0.0}


 67%|██████▋   | 2/3 [00:04<00:02,  2.33s/it]

{'loss': 3.0686, 'grad_norm': 56.99319839477539, 'learning_rate': 5e-06, 'epoch': 0.01}


100%|██████████| 3/3 [00:06<00:00,  2.26s/it]

{'loss': 3.8929, 'grad_norm': 54.15830612182617, 'learning_rate': 0.0, 'epoch': 0.01}


100%|██████████| 3/3 [00:07<00:00,  2.43s/it]

{'train_runtime': 7.2905, 'train_samples_per_second': 1.646, 'train_steps_per_second': 0.411, 'train_loss': 3.705877145131429, 'epoch': 0.01}


TrainOutput(global_step=3, training_loss=3.705877145131429, metrics={'train_runtime': 7.2905, 'train_samples_per_second': 1.646, 'train_steps_per_second': 0.411, 'total_flos': 269634011136.0, 'train_loss': 3.705877145131429, 'epoch': 0.009523809523809525})